In [70]:
import pandas as pd
import urllib.request
import numpy as np
from IPython.display import display
from sklearn.model_selection import StratifiedKFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing


In [53]:
print('Loading case data ...')
cases = pd.read_csv("pancancer_case_features.csv")
print("done.")

Loading case data ...


/Users/tonyd/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


done.


In [58]:
labels_string = cases.cancer_type
le = preprocessing.LabelEncoder()
labels = le.fit_transform(labels_string)

data = cases.loc[:,'age_at_diag':]

# Get the patient barcode.  This is what we will use to join the mutations to the clinical data
def parseAgeAtDiag(age_at_diag):
    if age_at_diag == '[Not Available]':
        return 0
    else:
        return age_at_diag

data['age_at_diag'] = data['age_at_diag'].apply(parseAgeAtDiag)


display(labels)
display(data.head())

array([ 8,  8,  8, ..., 28,  4,  3])

,age_at_diag,A2M,A2ML1,ABCA1,ABCA10,ABCA2,ABCA3,ABCA5,ABCA6,ABCA7,...,ZNF521,ZNF560,ZNF638,ZNF646,ZNF676,ZNF716,ZNF835,ZNF91,ZNFX1,ZZEF1
0,50,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,54,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,78,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,62,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,61,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [59]:
skf = StratifiedKFold(n_splits=10)
for train_index, test_index in skf.split(data, labels):
    train_data, test_data     = data.values[train_index], data.values[test_index]
    train_labels, test_labels = labels[train_index], labels[test_index]
    print(len(train_data), len(test_data))
    
display(train_data[0])


8994 1014
8998 1010
9001 1007
9003 1005
9005 1003
9007 1001
9011 997
9016 992
9016 992
9021 987


array([50, 0, 0, ..., 0, 0, 0], dtype=object)

In [ ]:
#
# Multinomial Naive Bayes
#
mnb = MultinomialNB()
params = {'alpha': [0.0001, 0.001, 0.01, 0.1, 0.5]}
mnb = GridSearchCV(mnb, params, cv=3,
                               scoring='accuracy', return_train_score=True)
# Fit  training data
mnb.fit(train_data, train_labels)  
# Show the best alpha parameter to use and the expected accuracy
print('\nMultinomial Naive Bayes Classifier')
print(' Best param:',   mnb.best_params_)
print(' Accuracy:  ',   np.round(mnb.best_score_, 4) )


#
# Logistic Regression
#
lr = LogisticRegression(penalty='l2', multi_class = 'ovr', solver='liblinear', max_iter=150)
params = {'C': [.01,  .18, .2]}
logit = GridSearchCV(lr, params, cv=2,
                     scoring='accuracy', return_train_score=True)

# Fit  training data
logit.fit(train_data, train_labels)  
# Show the best C parameter to use and the expected accuracy
print('Logistic Regression Classifier')
print(' Best param:', logit.best_params_)
print(' Accuracy:  ', np.round(logit.best_score_, 4) )




#
# KNN
#
knn = KNeighborsClassifier()
params = {'n_neighbors': [1,5]}
knn = GridSearchCV(knn, params, cv=2,
                           scoring='accuracy', return_train_score=True)
# Fit  training data
knn.fit(train_data, train_labels)  
# Show the best k_neighbors parameter to use and the expected accuracy
print('K Nearest Neighbor Classifier')
print(' Best param:', knn.best_params_)
print(' Accuracy:  ', np.round(knn.best_score_, 4) )





Multinomial Naive Bayes Classifier
 Best param: {'alpha': 0.1}
 Accuracy:   0.2574
